The first part of the analysis is to find the main predictor for each country

In [14]:
from utils import *
import numpy as np
import statsmodels.api as sm

# REDUCED = None
REDUCED = 2e5

# Preprocessed ratings 
preprocessed_ratings_ba = load_data("pre_ba", REDUCED)
preprocessed_ratings_ba['same_country'] = (preprocessed_ratings_ba['user_location'] == preprocessed_ratings_ba['brewery_location']).astype(int)


In [15]:
preprocessed_ratings_ba = pd.get_dummies(preprocessed_ratings_ba,columns=['style', 'user_location', 'brewery_location'], dtype='int')

In [16]:
columns_of_interest = ['abv','same_country']
columns_of_interest += [col for col in preprocessed_ratings_ba.columns if ('style_' in col or 'location_' in col)] 

In [17]:
X = preprocessed_ratings_ba[columns_of_interest]  # Predictors
y = preprocessed_ratings_ba['zscore']             # Target variable
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 zscore   R-squared:                       0.158
Model:                            OLS   Adj. R-squared:                  0.157
Method:                 Least Squares   F-statistic:                     144.0
Date:                Fri, 15 Dec 2023   Prob (F-statistic):               0.00
Time:                        13:47:40   Log-Likelihood:            -2.5299e+05
No. Observations:              200000   AIC:                         5.065e+05
Df Residuals:                  199738   BIC:                         5.092e+05
Df Model:                         261                                         
Covariance Type:            nonrobust                                         
                                                   coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf


def get_LR(data, columns):
    data_to_process = data.copy() # copy original dataset
    
    # create formula
    columns=list(columns)
    formula = 'rating ~ ' + columns[0]
    for el in columns[1:-1]:
        formula += ' + ' + el
    
    # standardization and creation of the formula
    columns.append('rating')  # add rating for the linear regression and standardization
    data_to_process = data_to_process[columns].dropna().sample(frac=1)  # only keeps columns of interest and shuffle the samples
    data_to_process = (data_to_process - data_to_process.mean()) / data_to_process.std()
    
    # create the model and fit it to the dataset
    mod = smf.ols(formula=formula, data=data_to_process)
    np.random.seed(2)
    res = mod.fit()
    return res

# Merge the datasets using the "user_id" column
merged_data = pd.merge(ratings_ba, users_ba[['user_id', 'location']], on='user_id', how='inner')

# Create an empty DataFrame to store regression results
results = pd.DataFrame(columns=['country', 'main_predictor'])

# Define the columns of interest (predictors)
columns_of_interest = ["appearance", "aroma", "palate", "taste", "overall"]

# Loop through each country and perform linear regression
for country in merged_data['location'].unique():
    country_data = merged_data[merged_data['location'] == country]
    try:
        res=get_LR(country_data, columns_of_interest)
        main_predictor = res.params.idxmax()  # Get the main predictor with the highest coefficient
        results = pd.concat([results, pd.DataFrame({'country': [country], 'main_predictor': [main_predictor]})], ignore_index=True)
    except:
        pass



# Load a world shapefile for mapping (you may need to download a suitable shapefile)
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Merge the world shapefile with the results DataFrame
world = world.merge(results, left_on='name', right_on='country', how='left')

# Plot the main predictor on a world map
fig, ax = plt.subplots(1, 1, figsize=(15, 10))
world.boundary.plot(ax=ax, linewidth=1)
world.plot(column='main_predictor', cmap='coolwarm', ax=ax, legend=True)
plt.title('Main Predictor by Country')
plt.show()
